In [1]:
import pandas as pd
import boto3
import numpy as np

In [2]:
bucket = 'datascience-sagemaker-laurawindlin'

In [5]:
dados_brutos_arquivo = 'dados_ac4.csv'

try:
    os.mkdir('dados')
except:
    pass

dados = pd.read_csv("s3://{}/{}".format(bucket, "dados_ac4.csv"))
pd.set_option('display.max_rows', 20) 
dados

,ID,Reason for absence,Month of absence,Day of the week,Seasons,Transportation expense,Distance from Residence to Work,Service time,Age,Work load Average/day,...,Disciplinary failure,Education,Son,Social drinker,Social smoker,Pet,Weight,Height,Body mass index,Absenteeism time in hours
0,11,26,7,3,1,289,36,13,33,239.554,...,0,1,2,1,0,1,90,172,30,4
1,36,0,7,3,1,118,13,18,50,239.554,...,1,1,1,1,0,0,98,178,31,0
2,3,23,7,4,1,179,51,18,38,239.554,...,0,1,0,1,0,0,89,170,31,2
3,7,7,7,5,1,279,5,14,39,239.554,...,0,1,2,1,1,0,68,168,24,4
4,11,23,7,5,1,289,36,13,33,239.554,...,0,1,2,1,0,1,90,172,30,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
735,11,14,7,3,1,289,36,13,33,264.604,...,0,1,2,1,0,1,90,172,30,8
736,1,11,7,3,1,235,11,14,37,264.604,...,0,3,1,0,0,1,88,172,29,4
737,4,0,0,3,1,118,14,13,40,271.219,...,0,1,1,1,0,8,98,170,34,0
738,8,0,0,4,2,231,35,14,39,271.219,...,0,1,2,1,0,2,100,170,35,0


In [ ]:
dados = dados.drop(['Transportation expense', 'Service time', 'Work load Average/day ', 'Reason for absence', 'Month of absence', 'Day of the week', 'Seasons', 'Hit target', 'Weight','Height', 'Body mass index'], axis=1) 

In [10]:
dados['ALVO'] = dados['Absenteeism time in hours'].apply(lambda x: 1 if x > 0  else 0) 

In [11]:
dados['FILHOS'] = dados['Son'].apply(lambda x: 1 if x > 0  else 0) 

In [12]:
dados['PETS'] = dados['Pet'].apply(lambda x: 1 if x > 0  else 0) 

In [13]:
dados['IDADE_NORM'] = dados['Age'].apply(lambda x: 28 if np.isnan(x) or x < 28 else x) # mínimo
dados['IDADE_NORM'] = dados['IDADE_NORM'].apply(lambda x: 50 if x > 50 else x) # máximo
dados['IDADE_NORM'] = dados['IDADE_NORM'].apply(lambda x: (x-28)/(50-28)) # normalização entre 0 e 1

In [14]:
dados['ESCOLARIDADE'] = dados['Education'].apply(lambda x: 1 if x > 1  else 0) 

In [15]:
dados['DISTÂNCIA'] = dados['Distance from Residence to Work'].apply(lambda x: (x-5)/(52-5)) # normalização entre 0 e 1

In [16]:
dados

,ID,Distance from Residence to Work,Age,Disciplinary failure,Education,Son,Social drinker,Social smoker,Pet,Absenteeism time in hours,ALVO,FILHOS,PETS,IDADE_NORM,ESCOLARIDADE,DISTÂNCIA
0,11,36,33,0,1,2,1,0,1,4,1,1,1,0.227273,0,0.659574
1,36,13,50,1,1,1,1,0,0,0,0,1,0,1.000000,0,0.170213
2,3,51,38,0,1,0,1,0,0,2,1,0,0,0.454545,0,0.978723
3,7,5,39,0,1,2,1,1,0,4,1,1,0,0.500000,0,0.000000
4,11,36,33,0,1,2,1,0,1,2,1,1,1,0.227273,0,0.659574
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
735,11,36,33,0,1,2,1,0,1,8,1,1,1,0.227273,0,0.659574
736,1,11,37,0,3,1,0,0,1,4,1,1,1,0.409091,1,0.127660
737,4,14,40,0,1,1,1,0,8,0,0,1,1,0.545455,0,0.191489
738,8,35,39,0,1,2,1,0,2,0,0,1,1,0.500000,0,0.638298


In [ ]:
dados = dados.drop(['Age', 'Education', 'Distance from Residence to Work', 'Son', 'Absenteeism time in hours', 'ID'], axis=1)

In [18]:
dados = dados.drop(['Pet'], axis=1)
dados

,Disciplinary failure,Social drinker,Social smoker,ALVO,FILHOS,PETS,IDADE_NORM,ESCOLARIDADE,DISTÂNCIA
0,0,1,0,1,1,1,0.227273,0,0.659574
1,1,1,0,0,1,0,1.000000,0,0.170213
2,0,1,0,1,0,0,0.454545,0,0.978723
3,0,1,1,1,1,0,0.500000,0,0.000000
4,0,1,0,1,1,1,0.227273,0,0.659574
...,...,...,...,...,...,...,...,...,...
735,0,1,0,1,1,1,0.227273,0,0.659574
736,0,0,0,1,1,1,0.409091,1,0.127660
737,0,1,0,0,1,1,0.545455,0,0.191489
738,0,1,0,0,1,1,0.500000,0,0.638298


In [ ]:
#Separação em treino (70%), validação(20%) e testes (10%)

In [19]:
treinamento, validacao, teste = np.split(dados.sample(frac=1, random_state=88), [int(0.7 * len(dados)), int(0.9 * len(dados))]) 
len(dados),len(treinamento), len(validacao), len(teste)

(740, 518, 148, 74)

In [20]:
treinamento.to_csv('dados/treinamento.csv', index=False, header=False)
validacao.to_csv('dados/validacao.csv', index=False, header=False)
teste.to_csv('dados/teste.csv', index=False)

In [ ]:
#Salvando no s3

In [ ]:
boto3.Session().resource('s3').Bucket(bucket).Object('treinamento.csv').upload_file('dados/treinamento.csv')
boto3.Session().resource('s3').Bucket(bucket).Object('validacao.csv').upload_file('dados/validacao.csv')
boto3.Session().resource('s3').Bucket(bucket).Object('teste.csv').upload_file('dados/teste.csv')